# Notebook to predict patient arrivals at HUSE
This jupyter notebook contains all the code used in the paper "Forecasting emergency department visits in the reference hospital of the Balearic Islands: the role of tourist and weather data."

In [ ]:
# Importing Python modules (Python version 3.11.4)

import numpy as np  # version 1.25.2
import pandas as pd  # version 2.1.1
import matplotlib.pyplot as plt  # version 3.7.1
from matplotlib import cm  # version 3.7.1
from datetime import datetime, timedelta  # version 3.11.4 (python base module)
import pickle  # version 3.11.4
import os  # version 3.11.4
import warnings  # version 3.11.4
from tqdm import tqdm  # version 4.65.0
from workalendar.europe import Spain  # version 17.0.0
from scipy.stats import multivariate_normal, norm  # version 1.11.1
from itertools import combinations, combinations_with_replacement # version 3.11.4
from sklearn.preprocessing import StandardScaler  # version 1.2.2
from sklearn.model_selection import train_test_split  # version 1.2.2
from sklearn.ensemble import RandomForestRegressor  # version 1.2.2
from sklearn.svm import SVR  # version 1.2.2
import torch  # version 2.0.1
import torch.nn as nn  # version 2.0.1
from dieboldmariano import dm_test  # version 1.1.0
from statsmodels.tsa.arima.model import ARIMA  # version 0.14.0

# Setting matplotlib graphic theme
plt.style.use('bmh')

# Defining time variables
ZERO_DATE = datetime(year=2015, month=12, day=26)
COVZ, COVE = 1527, 2197
COVID_ZERO = ZERO_DATE + pd.Timedelta(days=COVZ)
COVID_END = ZERO_DATE + pd.Timedelta(days=COVE)
END_STEP = 2562

# Setting the random seed
SEED = 14041999
np.random.seed(SEED)

# Defining age cohorts and sex
AGE_COHORTS = [0, 15, 25, 35, 45, 55, 65, 75, 85, np.inf]
AGE_LABELS = ['0-15', *[f"{i}-{i+10}" for i in range(15, 85, 10)], '85+']
GENDER_DICT = {'M': 'male', 'F': 'female', 'N': 'other'}

# Defining shifts and custom holidays
DAYTIMES = ['morning', 'afternoon', 'night']
PALMADAYS = [[6, 1], [20, 1], [1, 3], [1, 5], [15, 8],
             [12, 10], [1, 11], [6, 12], [8, 12], [26, 12]]

In [ ]:
# Importing the datasets
# They are defined UPPERCASE to distinguish them from the locally defined df

X_TRAIN = pd.read_csv('X_train.csv')
Y_TRAIN = pd.read_csv('Y_train.csv')
X_VALID = pd.read_csv('X_validation.csv')
Y_VALID = pd.read_csv('Y_validation.csv')
X_TEST = pd.read_csv('X_test.csv')
Y_TEST = pd.read_csv('Y_test.csv')
X_COVID = pd.read_csv('X_post-covid.csv')
Y_COVID = pd.read_csv('Y_post-covid.csv')

print('Datasets uploaded!')

In [ ]:
# Generating the week-month NIP plots
def weeklyEmergency():
    weeklabels = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
    monthlabels = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',
                   'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
    lstys = ['--', '--', '--', '--', '--', '-', '-']
    maks = ['o', 's', '^', 'd', 'v', 'o', 's']
    ylims = {'morning': [120, 210], 'afternoon': [80, 145], 'night': [50, 100]}
    XDF = pd.concat((X_TRAIN, X_VALID, X_TEST, X_COVID))
    YDF = pd.concat((Y_TRAIN, Y_VALID, Y_TEST, Y_COVID))
    for dt in DAYTIMES:
        figa, ax = plt.subplots(1, 1, figsize=(9, 7))
        for w in range(7):
            T = XDF[XDF['weekday'] == w]['timestep'].values
            months = [(ZERO_DATE + timedelta(days=t)).month for t in T]
            Y = YDF[XDF['weekday'] == w][f"total_{dt}"]
            xmonths = np.arange(1, 13)
            ymonths = np.zeros(12)
            for x in xmonths:
                ymonths[x-1] = np.mean(Y[months == x])
            ax.scatter(xmonths, ymonths, color=cm.Spectral_r(w/6), marker=maks[w],
                       label=weeklabels[w], linewidth=0.5, edgecolor='black',
                       s=100, zorder=24)
            ax.plot(xmonths, ymonths, color=cm.Spectral_r(w/6),
                    linestyle=lstys[w], zorder=12)
        ax.set_xticks(xmonths)
        ax.set_xticklabels(monthlabels, fontsize=14)
        ax.set_yticklabels(ax.get_yticks().astype(int), fontsize=14)
        ax.set_title(f"Average NIP per specific weekday/month ({dt} shift)", fontsize=16)
        ax.set_xlim(0.75, 12.25)
        ax.set_ylim(*ylims[dt])
        legend = ax.legend(loc='upper center', fontsize=13.5, ncol=7, columnspacing=0.5)
        legend.set_zorder(50)
        plt.savefig(f"weeks-vs-months_{dt}.pdf", bbox_inches='tight')
        plt.show()
        plt.close(figa)
            
            
weeklyEmergency()

### Hyperparameter tuning

In [ ]:
# performance function
def perforMeter(pred_y, true_y, shifts=DAYTIMES):
    diff = pred_y - true_y
    mae = np.mean(np.abs(diff), axis=0)
    rmse = np.sqrt(np.mean((diff) ** 2, axis=0))
    mape = 200 * np.mean(np.abs(diff) / (np.abs(pred_y) + np.abs(true_y)), axis=0)
    dicto = {'mape': {}, 'rmse': {}, 'mae': {}}
    for i, dt in enumerate(shifts):
        dicto['mape'][dt] = mape[i]
        dicto['rmse'][dt] = rmse[i]
        dicto['mae'][dt] = mae[i]
    return dicto


# Hyperparameter tuning: Random Forests
def treeTuner(X_train, X_valid, Y_train, Y_valid, ntrees, niters=100):
    columns = ['n_trees', 'shift', 'mape', 'mape_sd', 'rmse', 'rmse_sd', 'mae', 'mae_sd']
    Y_train = Y_train.values
    X_tr = X_train.values
    X_te = X_valid.values
    normies = [StandardScaler() for _ in range(X_tr.shape[1])]
    for i in range(len(normies)):
        normies[i].fit(X_tr[:, i].reshape(-1, 1))
        X_tr[:, i] = normies[i].transform(X_tr[:, i].reshape(-1, 1)).flatten()
        X_te[:, i] = normies[i].transform(X_te[:, i].reshape(-1, 1)).flatten()
    df = []
    for nt in ntrees:
        subdict = {'mape': dict(zip(DAYTIMES, [[], [], []])),
                   'rmse': dict(zip(DAYTIMES, [[], [], []])),
                   'mae': dict(zip(DAYTIMES, [[], [], []]))}
        woods = RandomForestRegressor(n_estimators=nt, n_jobs=-1)
        woods.fit(X_tr, Y_train)
        full_preds = woods.predict(X_te)
        for ni in tqdm(range(niters), colour='green', ncols=111, desc=f"{nt}",
                       position=0, leave=True):
            if ni % 10 == 0:
                np.random.seed(SEED + ni // 10)
            vidx = np.random.randint(low=0, high=X_valid.shape[0], size=X_valid.shape[0])
            tree_preds = full_preds.copy()[vidx]
            Y_test = Y_valid.values.copy()[vidx]
            dicto = perforMeter(tree_preds, Y_test)
            for met in subdict:
                for dt in dicto[met]:
                    subdict[met][dt].append(dicto[met][dt])
        for dt in DAYTIMES:
            line = [nt, dt]
            for meth in subdict:
                line.append(np.mean(subdict[meth][dt]))
                line.append(np.std(subdict[meth][dt]))
            df.append(line)
    df = pd.DataFrame(df, columns=columns)
    df.to_csv(f"tuning-df_rf.csv", index=None)
    np.random.seed(SEED)
    display(df)


ycols = ['total_morning', 'total_afternoon', 'total_night']
tree_axis = np.arange(25, 201, 25)
treeTuner(X_TRAIN, X_VALID, Y_TRAIN[ycols], Y_VALID[ycols], tree_axis, niters=1000)

In [ ]:
# performance function
def perforMeter(pred_y, true_y, shifts=DAYTIMES):
    diff = pred_y - true_y
    mae = np.mean(np.abs(diff), axis=0)
    rmse = np.sqrt(np.mean((diff) ** 2, axis=0))
    mape = 200 * np.mean(np.abs(diff) / (np.abs(pred_y) + np.abs(true_y)), axis=0)
    dicto = {'mape': {}, 'rmse': {}, 'mae': {}}
    for i, dt in enumerate(shifts):
        dicto['mape'][dt] = mape[i]
        dicto['rmse'][dt] = rmse[i]
        dicto['mae'][dt] = mae[i]
    return dicto


# Hyperparameter tuning: Support Vector Regressor
def vecTuner(X_train, X_valid, Y_train, Y_valid, degrees, niters=100):
    columns = ['degree', 'shift', 'mape', 'mape_sd', 'rmse', 'rmse_sd', 'mae', 'mae_sd']
    Y_train = Y_train.values
    df = []
    X_tr = X_train.values
    X_te = X_valid.values
    normies = [StandardScaler() for _ in range(X_tr.shape[1])]
    for i in range(len(normies)):
        normies[i].fit(X_tr[:, i].reshape(-1, 1))
        X_tr[:, i] = normies[i].transform(X_tr[:, i].reshape(-1, 1)).flatten()
        X_te[:, i] = normies[i].transform(X_te[:, i].reshape(-1, 1)).flatten()
    for nt in degrees:
        subdict = {'mape': dict(zip(DAYTIMES, [[], [], []])),
                   'rmse': dict(zip(DAYTIMES, [[], [], []])),
                   'mae': dict(zip(DAYTIMES, [[], [], []]))}
        svr = [SVR(degree=nt) for _ in range(3)]
        full_preds = []
        for i in range(3):
            svr[i].fit(X_tr, Y_train[:, i])
            full_preds.append(svr[i].predict(X_te))
        full_preds = np.array(full_preds).T
        for ni in tqdm(range(niters), colour='green', ncols=111, desc=f"{nt}",
                       position=0, leave=True):
            if ni % 10 == 0:
                np.random.seed(SEED + ni // 10)
            vidx = np.random.randint(low=0, high=X_valid.shape[0], size=X_valid.shape[0])
            svr_preds = full_preds.copy()[vidx]
            Y_test = Y_valid.values[vidx]
            dicto = perforMeter(svr_preds, Y_test)
            for met in subdict:
                for dt in dicto[met]:
                    subdict[met][dt].append(dicto[met][dt])
        for dt in DAYTIMES:
            line = [nt, dt]
            for meth in subdict:
                line.append(np.mean(subdict[meth][dt]))
                line.append(np.std(subdict[meth][dt]))
            df.append(line)
    df = pd.DataFrame(df, columns=columns)
    df.to_csv(f"tuning-df_svr.csv", index=None)
    np.random.seed(SEED)
    display(df)


ycols = ['total_morning', 'total_afternoon', 'total_night']
deg_axis = np.arange(1, 11)
vecTuner(X_TRAIN, X_VALID, Y_TRAIN[ycols], Y_VALID[ycols], deg_axis, niters=1000)

In [ ]:
# Plotting the results of hyperparameter tuning for RF and SVR
def simpleTuPlotter(chosen_rf, chosen_svr, sigmas=1):
    tdf = pd.read_csv(f"tuning-df_rf.csv")
    vdf = pd.read_csv(f"tuning-df_svr.csv")
    colors = dict(zip(DAYTIMES, ['xkcd:kermit green', 'xkcd:scarlet', 'xkcd:sapphire']))
    figa, ax = plt.subplots(1, 2, figsize=(14, 6))
    for shift in DAYTIMES:
        stdf = tdf[tdf['shift'] == shift]
        svdf = vdf[vdf['shift'] == shift]
        ax[0].plot(stdf['n_trees'], stdf['mape'], marker='o', color=colors[shift],
                   label=shift, zorder=10)
        ax[0].fill_between(stdf['n_trees'], stdf['mape'] + sigmas * stdf['mape_sd'],
                           stdf['mape'] - sigmas * stdf['mape_sd'], color=colors[shift],
                           zorder=9, alpha=0.25)
        ax[1].plot(svdf['degree'], svdf['mape'], marker='o', color=colors[shift],
                   label=shift, zorder=10)
        ax[1].fill_between(svdf['degree'], svdf['mape'] + sigmas * svdf['mape_sd'],
                           svdf['mape'] - sigmas * svdf['mape_sd'], color=colors[shift],
                           zorder=9, alpha=0.25)
    ax[0].axvline(chosen_rf, linestyle='--', color='xkcd:steel gray')
    ax[1].axvline(chosen_svr, linestyle='--', color='xkcd:steel gray')
    ax[0].set_xticks(range(25, 201, 25))
    ax[1].set_xticks(range(1, 11))
    ax[0].set_xlabel('Number of trees', fontsize=12)
    ax[1].set_xlabel('Degree', fontsize=12)
    ax[0].set_title('Random Forest')
    ax[1].set_title('Support Vector Regressor')
    for i in range(2):
        ax[i].legend(loc='upper right', fontsize=12)
        ax[i].set_ylabel('MAPE on validation dataset', fontsize=12)
        ax[i].set_yticks(range(0, 25, 1))
        ax[i].set_ylim(6, 20)
    plt.savefig(f"tuning-plot_rf-svf.pdf", bbox_inches='tight')
    plt.show()
    
    
simpleTuPlotter(100, 3, sigmas=1)

In [ ]:
# Hyperparameter tuning: Feedforward Neural Network
def deepTuner(X_train, X_valid, Y_train, Y_valid, nepochs, niters=100):
    L, VL = [], []
    X_tr = X_train.values
    y = torch.Tensor(Y_train.values)
    normies = [StandardScaler() for _ in range(X_tr.shape[1])]
    for i in range(len(normies)):
        normies[i].fit(X_tr[:, i].reshape(-1, 1))
        X_tr[:, i] = normies[i].transform(X_tr[:, i].reshape(-1, 1)).flatten()
    X = torch.Tensor(X_tr)
    crisafully_cnet = nn.Sequential(nn.Linear(X.shape[1], 32), nn.ReLU(),
                                    nn.Linear(32, 64), nn.ReLU(),
                                    nn.Linear(64, 128), nn.ReLU(),
                                    nn.Linear(128, 256), nn.ReLU(),
                                    nn.Linear(256, 512), nn.ReLU(),
                                    nn.Linear(512, 1024), nn.ReLU(),
                                    nn.Linear(1024, 2048), nn.ReLU(),
                                    nn.Linear(2048, 1024), nn.ReLU(),
                                    nn.Linear(1024, 512), nn.ReLU(),
                                    nn.Linear(512, 256), nn.ReLU(),
                                    nn.Linear(256, 128), nn.ReLU(),
                                    nn.Linear(128, 64), nn.ReLU(),
                                    nn.Linear(64, 32), nn.ReLU(),
                                    nn.Linear(32, y.shape[1]))
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(crisafully_cnet.parameters(), lr=0.01)
    for epoch in tqdm(range(nepochs), colour='green', ncols=111,
               position=0, leave=True):
        vlosses = []
        optimizer.zero_grad()
        output = crisafully_cnet(X)
        loss = criterion(output, y)
        L.append(loss.item())
        with torch.no_grad():
            for ni in range(niters):
                if ni % 10 == 0:
                    np.random.seed(SEED + ni // 10)
                vidx = np.random.randint(low=0, high=X_valid.shape[0], size=X_valid.shape[0])
                X_te = X_valid.values.copy()[vidx]
                for i in range(len(normies)):
                    X_te[:, i] = normies[i].transform(X_te[:, i].reshape(-1, 1)).flatten()
                Xv = torch.Tensor(X_te)
                yv = torch.Tensor(Y_valid.values.copy()[vidx])
                output = crisafully_cnet(Xv)
                vloss = criterion(output, yv)
                vlosses.append(vloss.item())
        loss.backward()
        optimizer.step()
        VL.append(vlosses)
    np.random.seed(SEED)
    pickle.dump({'L': np.array(L), 'VL': np.array(VL)},
                open("tuning-dict_fnn.pickle", 'wb'))


ycols = ['total_morning', 'total_afternoon', 'total_night']
deepTuner(X_TRAIN, X_VALID, Y_TRAIN[ycols], Y_VALID[ycols], 500, niters=1000)

In [ ]:
# Plotting the results of hyperparameter tuning for FNN
def deepTuPlotter(L, VL, chosen, ylim=None, xlim=None):
    losses, vlosses = L, np.mean(VL, axis=1)
    slosses = np.std(VL, axis=1)
    nepochs = len(losses)
    if xlim is None:
        xlim = nepochs
    figa, ax = plt.subplots(1, 1, figsize=(8, 6))
    ax.plot(range(nepochs), losses, zorder=10,
            label='training set', color='xkcd:sapphire')
    ax.fill_between(range(nepochs), vlosses + slosses, vlosses - slosses,
                    color='xkcd:scarlet', alpha=0.5, zorder=1)
    ax.plot(range(nepochs), vlosses, zorder=9,
            label='validation set', color='xkcd:scarlet')
    ax.axvline(chosen, linestyle='--', color='xkcd:steel gray')
    ax.set_ylim(0, ylim)
    ax.set_xlim(0, xlim)
    ax.set_xlabel('Number of epochs', fontsize=14)
    ax.set_ylabel('Loss function', fontsize=14)
    ax.legend(loc='upper right', fontsize=12)
    plt.savefig(f"tuning-plot_fnn.pdf", bbox_inches='tight')
    plt.show()


loss_dict = pickle.load(open("tuning-dict_fnn.pickle", 'rb'))
deepTuPlotter(loss_dict['L'], loss_dict['VL'], chosen=130, ylim=1000, xlim=500)

### Predictions for the test dataset

In [ ]:
# Generating predictions for the test dataset for SARIMA, RF, SVR, and FNN
def theGreatPredictor(X_train, X_test, Y_train, Y_test, Y_valid, ntrees, degree, nepochs,
                      niters=100, aged=False):
    T = {}
    wp_cols = ['timestep', 'weekday', 'yearday', 'holiday_-2',
               'holiday_-1', 'holiday_0', 'holiday_+1', 'holiday_+2',
               'temp_min', 'temp_max', 'prec_prob', 'wind_speed',
               'resident_pop', 'tourist_pop']
    wx_cols = ['timestep', 'weekday', 'yearday', 'holiday_-2',
               'holiday_-1', 'holiday_0', 'holiday_+1', 'holiday_+2',
               'temp_min', 'temp_max', 'prec_prob', 'wind_speed', 'resident_pop']
    xp_cols = ['timestep', 'weekday', 'yearday', 'holiday_-2',
               'holiday_-1', 'holiday_0', 'holiday_+1', 'holiday_+2',
               'resident_pop', 'tourist_pop']
    xx_cols = ['timestep', 'weekday', 'yearday', 'holiday_-2',
               'holiday_-1', 'holiday_0', 'holiday_+1', 'holiday_+2', 'resident_pop']
    if aged:
        daytimes = ['low', 'medium', 'high']
        prefix = 'risk'
    else:
        daytimes = DAYTIMES
        prefix = 'shift'
    # SARIMA
    Y_train, Y_valid, Y_test = Y_train.values, Y_valid.values, Y_test
    Y_a1 = np.zeros((len(Y_test), 3))
    Y_a3 = np.zeros((len(Y_test), 3))
    Y_a7 = np.zeros((len(Y_test), 3))
    Y_a14 = np.zeros((len(Y_test), 3))
    for t in tqdm(np.arange(-14, len(Y_test)-1), colour='red', ncols=111, desc='SARIMA'):
        if t < -1:
            Y = Y_valid[:t+1]
        elif t == -1:
            Y = Y_valid
        else:
            Y = np.row_stack((Y_valid[t+1:], Y_test[:t+1]))
        with warnings.catch_warnings(action="ignore"):
            arima = [ARIMA(Y[:, i], order=(1, 1, 1), seasonal_order=(1, 1, 1, 7),
                           ).fit() for i in range(3)]
        if t < len(Y_test) - 14:
            apred = np.array([arima[i].forecast(steps=14) for i in range(3)])
            if t > -2:
                Y_a1[t+1] = apred[:, 0]
            if t > -4:
                Y_a3[t+3] = apred[:, 2]
            if t > -8:
                Y_a7[t+7] = apred[:, 6]
            Y_a14[t+14] = apred[:, 13]
        if t < len(Y_test) - 7:
            apred = np.array([arima[i].forecast(steps=7) for i in range(3)])
            Y_a1[t+1] = apred[:, 0]
            Y_a3[t+3] = apred[:, 2]
            Y_a7[t+7] = apred[:, 6]
        elif t < len(Y_test) - 3:
            apred = np.array([arima[i].forecast(steps=3) for i in range(3)])
            Y_a1[t+1] = apred[:, 0]
            Y_a3[t+3] = apred[:, 2]
        else:
            apred = np.array([arima[i].forecast(steps=1)[0] for i in range(3)])
            Y_a1[t+1] = apred
    T['SARIMA-1'] = dict(zip(daytimes, np.array([Y_a1[:, t] for t in range(3)])))
    T['SARIMA-3'] = dict(zip(daytimes, np.array([Y_a3[:, t] for t in range(3)])))
    T['SARIMA-7'] = dict(zip(daytimes, np.array([Y_a7[:, t] for t in range(3)])))
    T['SARIMA-14'] = dict(zip(daytimes, np.array([Y_a14[:, t] for t in range(3)])))
    # RF, SVR, and FNN
    X_tr = {'CWP': X_train[wp_cols].copy().values, 'CWX': X_train[wx_cols].copy().values,
            'CXP': X_train[xp_cols].copy().values, 'CXX': X_train[xx_cols].copy().values}
    T['RF'] = {}
    T['SVR'] = {}
    T['FNN'] = {}
    normies, woods, svr = {}, {}, {}
    y = torch.Tensor(Y_train)
    for k, cols in zip(X_tr, [wp_cols, wx_cols, xp_cols, xx_cols]):
        normies[k] = [StandardScaler() for _ in range(X_tr[k].shape[1])]
        for i in range(len(normies[k])):
            normies[k][i].fit(X_tr[k][:, i].reshape(-1, 1))
            X_tr[k][:, i] = normies[k][i].transform(X_tr[k][:, i].reshape(-1, 1)).flatten()
        woods[k] = RandomForestRegressor(n_estimators=ntrees, n_jobs=-1) 
        woods[k].fit(X_tr[k], Y_train)
        svr[k] = [SVR(degree=degree) for _ in range(3)]
        for i in range(3):
            svr[k][i].fit(X_tr[k], Y_train[:, i])
        X = torch.Tensor(X_tr[k])
        crisafully_cnet = nn.Sequential(nn.Linear(X.shape[1], 32), nn.ReLU(),
                                        nn.Linear(32, 64), nn.ReLU(),
                                        nn.Linear(64, 128), nn.ReLU(),
                                        nn.Linear(128, 256), nn.ReLU(),
                                        nn.Linear(256, 512), nn.ReLU(),
                                        nn.Linear(512, 1024), nn.ReLU(),
                                        nn.Linear(1024, 2048), nn.ReLU(),
                                        nn.Linear(2048, 1024), nn.ReLU(),
                                        nn.Linear(1024, 512), nn.ReLU(),
                                        nn.Linear(512, 256), nn.ReLU(),
                                        nn.Linear(256, 128), nn.ReLU(),
                                        nn.Linear(128, 64), nn.ReLU(),
                                        nn.Linear(64, 32), nn.ReLU(),
                                        nn.Linear(32, y.shape[1]))
        criterion = nn.MSELoss()
        optimizer = torch.optim.Adam(crisafully_cnet.parameters(), lr=0.01)
        for epoch in tqdm(range(nepochs), colour='blue', ncols=111,
                          position=0, leave=True, desc=f'FNN training ({k})'):
            optimizer.zero_grad()
            output = crisafully_cnet(X)
            loss = criterion(output, y)
            loss.backward()
            optimizer.step()
        T['RF'][k] = {}
        T['SVR'][k] = {}
        T['FNN'][k] = {}
        with torch.no_grad():
            X_te = X_test[cols].copy().values
            for i in range(len(normies[k])):
                X_te[:, i] = normies[k][i].transform(X_te[:, i].reshape(-1, 1)).flatten()
            tree_preds = woods[k].predict(X_te)
            Xv = torch.Tensor(X_te)
            deep_preds = crisafully_cnet(Xv).numpy()
            for t, dt in enumerate(daytimes):
                T['RF'][k][dt] = tree_preds[:, t]
                T['SVR'][k][dt] = svr[k][t].predict(X_te)
                T['FNN'][k][dt] = deep_preds[:, t]
    # Bootstrap procedure
    D = {'Y': dict(zip(daytimes, [[], [], []])),
         'SARIMA-1': dict(zip(daytimes, [[], [], []])),
         'SARIMA-3': dict(zip(daytimes, [[], [], []])),
         'SARIMA-7': dict(zip(daytimes, [[], [], []])),
         'SARIMA-14': dict(zip(daytimes, [[], [], []])),
         'RF': dict(zip(X_tr, [dict(zip(daytimes, [[], [], []])) for _ in range(4)])),
         'SVR': dict(zip(X_tr, [dict(zip(daytimes, [[], [], []])) for _ in range(4)])),
         'FNN': dict(zip(X_tr, [dict(zip(daytimes, [[], [], []])) for _ in range(4)]))}
    for ni in tqdm(range(niters), colour='red', ncols=111, desc='Bootstrapping'):
        if ni % 10 == 0:
            np.random.seed(SEED + ni // 10)
        vidx = np.random.randint(low=0, high=X_test.shape[0], size=X_test.shape[0])
        for t, dt in enumerate(daytimes):
            D['Y'][dt].append(Y_test.values[vidx, t])
            D['SARIMA-1'][dt].append(T['SARIMA-1'][dt][vidx])
            D['SARIMA-3'][dt].append(T['SARIMA-3'][dt][vidx])
            D['SARIMA-7'][dt].append(T['SARIMA-7'][dt][vidx])
            D['SARIMA-14'][dt].append(T['SARIMA-14'][dt][vidx])
            for meth in ['RF', 'SVR', 'FNN']:
                for k in X_tr:
                    D[meth][k][dt].append(T[meth][k][dt][vidx])
    for t, dt in enumerate(daytimes):
        for meth in ['Y', 'SARIMA-1', 'SARIMA-3', 'SARIMA-7', 'SARIMA-14']:
            D[meth][dt] = np.array(D[meth][dt])
        for meth in ['RF', 'SVR', 'FNN']:
            for k in X_tr:
                D[meth][k][dt] = np.array(D[meth][k][dt])
    pickle.dump(D, open(f"great-predict_{prefix}.pickle", 'wb'))
    

# shift-based predictions
ycols = ['total_morning', 'total_afternoon', 'total_night']
theGreatPredictor(X_TRAIN, X_TEST, Y_TRAIN[ycols], Y_TEST[ycols], Y_VALID[ycols],
                  ntrees=100, degree=3, nepochs=130, niters=1000, aged=False)

# risk-group-based predictions
ycols = ['total_low', 'total_medium', 'total_high']
theGreatPredictor(X_TRAIN, X_TEST, Y_TRAIN[ycols], Y_TEST[ycols], Y_VALID[ycols],
                  ntrees=100, degree=3, nepochs=130, niters=1000, aged=True)

In [ ]:
# performance function
def perforMeter(pred_y, true_y, shifts=DAYTIMES):
    dicto = {'mape': {}, 'mape_std': {}, 'rmse': {},
             'rmse_std': {}, 'mae': {}, 'mae_std': {}}
    for t in shifts:
        diff = pred_y[t] - true_y[t]
        mape = 200 * np.mean(np.abs(diff) / (np.abs(pred_y[t]) + np.abs(true_y[t])), axis=1)
        rmse = np.sqrt(np.mean((diff) ** 2, axis=1))
        mae = np.mean(np.abs(diff), axis=1)
        dicto['mape'][t] = np.mean(mape)
        dicto['mape_std'][t] = np.std(mape)
        dicto['rmse'][t] = np.mean(rmse)
        dicto['rmse_std'][t] = np.std(rmse)
        dicto['mae'][t] = np.mean(mae)
        dicto['mae_std'][t] = np.std(mae)
    return dicto


# Generating supplementary tables
def resultFunction(GD, aged=False):
    if aged:
        daytimes = ['low', 'medium', 'high']
        prefix = 'risk'
    else:
        daytimes = DAYTIMES
        prefix = 'shift'
    niters = len(GD['SARIMA-1'])
    metrics = ['mape', 'rmse', 'mae']
    inpudict = {'CWP': 'All', 'CXP': 'No W', 'CWX': 'No T', 'CXX': 'No W No T'}
    columns = ['method', 'input', 'shift', 'mape', 'mape_std',
               'rmse', 'rmse_std', 'mae', 'mae_std']
    # performance table
    df = []
    for meth in ['SARIMA-1', 'SARIMA-7', 'SARIMA-14']:
        dicto = perforMeter(GD[meth], GD['Y'], shifts=daytimes)
        for t in daytimes:
            df.append([meth, '', t.capitalize(), dicto['mape'][t], dicto['mape_std'][t],
                       dicto['rmse'][t], dicto['rmse_std'][t], dicto['mae'][t],
                       dicto['mae_std'][t]])
    for meth in ['RF', 'SVR', 'FNN']:
        for ind in inpudict:
            dicto = perforMeter(GD[meth][ind], GD['Y'], shifts=daytimes)
            for t in daytimes:
                df.append([meth, inpudict[ind], t.capitalize(), dicto['mape'][t],
                           dicto['mape_std'][t], dicto['rmse'][t], dicto['rmse_std'][t],
                           dicto['mae'][t], dicto['mae_std'][t]])
    df = pd.DataFrame(df, columns=columns)
    df.to_csv(f"metric-table-full_{prefix}.csv", index=None)
    display(df)


# Generating main tables
def toLatex():
    sdf = pd.read_csv(f"metric-table-full_shift.csv")
    rdf = pd.read_csv(f"metric-table-full_risk.csv")
    for metric in ['mape', 'rmse', 'mae']:
        columns = ['method', 'input', f'{metric}_morning', f'{metric}_afternoon',
                   f'{metric}_night', f'{metric}_low', f'{metric}_medium', f'{metric}_high']
        df = [['SARIMA-1', float('nan'), *sdf[metric].iloc[:3], *rdf[metric].iloc[:3]],
              ['SARIMA-7', float('nan'), *sdf[metric].iloc[3:6], *rdf[metric].iloc[3:6]]]
        for i in range(6, 40, 3):
            df.append([sdf.iloc[i]['method'], sdf.iloc[i]['input'],
                       *sdf.iloc[i:i+3][metric], *rdf.iloc[i:i+3][metric]])
        df = pd.DataFrame(df, columns=columns)
        df.to_csv(f"metric-table-latex_{metric}.csv", index=None)
        display(df)


gdict = pickle.load(open(f"great-predict_shift.pickle", 'rb'))
resultFunction(gdict, aged=False)

gdict = pickle.load(open(f"great-predict_risk.pickle", 'rb'))
resultFunction(gdict, aged=True)

toLatex()

### Diebold-Mariano tests

In [ ]:
# Diebold-Mariano fraction function
def dieboFunc(pred_a, pred_b, y_test, shifts=DAYTIMES, thresh=0.05, n_tests=1):
    diebolds = {}
    corr = thresh / n_tests
    for t in shifts:
        diebolds[t] = [dm_test(y_test[t][ni], pred_a[t][ni], pred_b[t][ni])[1]
                       for ni in range(y_test[t].shape[0])]
        diebolds[t] = [dm > corr for dm in diebolds[t]]
        diebolds[t] = np.sum(diebolds[t]) / len(diebolds[t])
    return diebolds


# Diebold-Mariano table generator
def dieBolder(GD, aged=False, n_tests=1):
    if aged:
        daytimes = ['low', 'medium', 'high']
        prefix = 'risk'
    else:
        daytimes = DAYTIMES
        prefix = 'shift'
    inpudict = {'CWP': 'All', 'CXP': 'No W', 'CWX': 'No T', 'CXX': 'No W T'}
    # diebold table
    dbar = tqdm(total=n_tests, colour='yellow', ncols=111,
                position=0, leave=True, desc=f'die-{prefix}')
    df = []
    columns = ['method_1', 'input_1', 'method_2', 'input_2', 'shift', 'equiv_frac']
    for m1, m2 in combinations(['SARIMA-1', 'SARIMA-7', 'SARIMA-14'], 2):
        diebold = dieboFunc(GD[m1], GD[m2], GD['Y'],
                            shifts=daytimes, thresh=0.05, n_tests=n_tests)
        dbar.update(3)
        for t in daytimes:
            df.append([m1, '', m2, '', t.capitalize(), diebold[t]])
    for bmeth in ['SARIMA-1', 'SARIMA-7', 'SARIMA-14']:
        for meth in ['RF', 'SVR', 'FNN']:
            for ind in inpudict:
                diebold = dieboFunc(GD[bmeth], GD[meth][ind], GD['Y'],
                                    shifts=daytimes, thresh=0.05, n_tests=n_tests)
                dbar.update(3)
                for t in daytimes:
                    df.append([bmeth, '', meth, inpudict[ind], t.capitalize(), diebold[t]])
    for m1, m2 in combinations_with_replacement(['RF', 'SVR', 'FNN'], 2):
        for v1, v2 in combinations_with_replacement(['CWP', 'CWX', 'CXP', 'CXX'], 2):
            if m1 == m2 and v1 == v2:
                continue
            diebold = dieboFunc(GD[m1][v1], GD[m2][v2], GD['Y'],
                                shifts=daytimes, thresh=0.05, n_tests=n_tests)
            dbar.update(3)
            for t in daytimes:
                df.append([m1, inpudict[v1], m2, inpudict[v2], t.capitalize(), diebold[t]])
            if m1 != m2 and v1 != v2:
                diebold = dieboFunc(GD[m1][v2], GD[m2][v1], GD['Y'],
                                    shifts=daytimes, thresh=0.05, n_tests=n_tests)
                dbar.update(3)
                for t in daytimes:
                    df.append([m1, inpudict[v2], m2, inpudict[v1],
                               t.capitalize(), diebold[t]])
    df = pd.DataFrame(df, columns=columns)
    df.to_csv(f"diebold-table_{prefix}.csv", index=None)
    display(df)
    
    
gdict = pickle.load(open(f"great-predict_shift.pickle", 'rb'))
dieBolder(gdict, aged=False, n_tests=315)

gdict = pickle.load(open(f"great-predict_risk.pickle", 'rb'))
dieBolder(gdict, aged=True, n_tests=315)

In [ ]:
# Diebold-Mariano plot generator
def diePlotter(df, aged=False, thresh=0.95):
    if aged:
        daytimes = ['low', 'medium', 'high']
        prefix = 'risk'
        colors = ['xkcd:jade green', 'xkcd:tangerine', 'xkcd:scarlet']
    else:
        daytimes = DAYTIMES
        prefix = 'shift'
        colors = ['xkcd:kermit green', 'xkcd:scarlet', 'xkcd:sapphire']
    figa, ax = plt.subplots(1, 1, figsize=(8, 8))
    metcoordict = {'SARIMA-1': 0, 'SARIMA-7': 1, 'SARIMA-14': 2,
                   'RF': 3, 'SVR': 7, 'FNN': 11}
    varcoordict = dict(zip(['nan', 'All', 'No T', 'No W', 'No W T'], [0, *range(4)]))
    colordict = dict(zip(daytimes, colors))
    angledict = dict(zip(daytimes, [np.pi/6, np.pi*5/6, np.pi*3/2]))
    varcoordict[''] = 0
    varlabs = ['-All', '-No T', '-No W', '-No W T']
    axlabels = ['SARIMA-1', 'SARIMA-7', 'SARIMA-14',
                *[''.join([m, v]) for m in ['RF', 'SVR', 'FNN'] for v in varlabs]]
    for _, line in df.iterrows():
        x = metcoordict[line['method_1']] + varcoordict[str(line['input_1'])]
        y = metcoordict[line['method_2']] + varcoordict[str(line['input_2'])]
        if line['shift'].lower() == 'morning' or line['shift'].lower() == 'low':
            ax.scatter(x, y, color='white', edgecolor='black', zorder=9, s=400)
        if line['equiv_frac'] > thresh:
            ax.scatter(x + np.cos(angledict[line['shift'].lower()])*0.15,
                       y + np.sin(angledict[line['shift'].lower()])*0.15,
                       color=colordict[line['shift'].lower()], zorder=10,
                       edgecolor='black', s=50)
    for t in daytimes:
        ax.scatter([], [], color=colordict[t], zorder=10, edgecolor='black', s=50, label=t)
    ax.set_xticks(range(0, 15))
    ax.set_yticks(range(0, 15))
    ax.set_xticklabels(axlabels, rotation=90)
    ax.set_yticklabels(axlabels)
    ax.legend(loc='lower right', title='The model is equivalent\nwhen predicting:')
    ax.set_xlim(-1, 15)
    ax.set_ylim(-1, 15)
    plt.savefig(f"diebold-plot_{prefix}.pdf", bbox_inches='tight')
    plt.show()
    
    
for aged, prefix in [[False, 'shift'], [True, 'risk']]:
    die_df = pd.read_csv(f"diebold-table_{prefix}.csv")
    diePlotter(die_df, aged=aged, thresh=0.9)

### Generating Figures 5 and 6
We chose RF (No W.) as the optimal model, but the function can be adapted to any method and input variables

In [ ]:
# performance function
def perforMeter(pred_y, true_y, shifts=DAYTIMES):
    diff = pred_y - true_y
    mae = np.mean(np.abs(diff), axis=0)
    rmse = np.sqrt(np.mean((diff) ** 2, axis=0))
    mape = 200 * np.mean(np.abs(diff) / (np.abs(pred_y) + np.abs(true_y)), axis=0)
    dicto = {'mape': {}, 'rmse': {}, 'mae': {}}
    for i, dt in enumerate(shifts):
        dicto['mape'][dt] = mape[i]
        dicto['rmse'][dt] = rmse[i]
        dicto['mae'][dt] = mae[i]
    return dicto


# Testing the model on the test dataset without bootstrap
def timeLiner(X_train, X_test, Y_train, Y_test, aged=False, xres=7, parts=5, ntrees=100):
    tlpath = f"timelines_rf-opt/"
    
    if not os.path.exists(tlpath):
        os.makedirs(tlpath)
    if aged:
        daytimes = ['low', 'medium', 'high']
        colors = ['xkcd:jade green', 'xkcd:tangerine', 'xkcd:scarlet']
        prefix = 'risk'
        ylim = 350
    else:
        daytimes = DAYTIMES
        colors = ['xkcd:kermit green', 'xkcd:scarlet', 'xkcd:sapphire']
        prefix = 'shift'
        ylim = 275
    cols = ['timestep', 'weekday', 'yearday', 'holiday_-2',
           'holiday_-1', 'holiday_0', 'holiday_+1', 'holiday_+2',
           'resident_pop', 'tourist_pop']
    meth = 'RF-No W'
    xaxis = X_test['timestep'].values
    nx = xaxis.shape[0]
    Y_train, Y_test = Y_train.values, Y_test.values
    X_train = X_train[cols].values
    X_test = X_test[cols].values
    normies = [StandardScaler() for _ in range(X_train.shape[1])]
    for i in range(len(normies)):
        normies[i].fit(X_train[:, i].reshape(-1, 1))
        X_train[:, i] = normies[i].transform(X_train[:, i].reshape(-1, 1)).flatten()
        X_test[:, i] = normies[i].transform(X_test[:, i].reshape(-1, 1)).flatten()
    woods = RandomForestRegressor(n_estimators=ntrees, n_jobs=-1) 
    woods.fit(X_train, Y_train)
    Y_pred = woods.predict(X_test)
    rmse = perforMeter(Y_pred, Y_test, shifts=daytimes)['rmse']
    for part in range(1, parts + 1):
        figa, ax = plt.subplots(1, 1, figsize=(12, 6))
        for ci in range(3):
            ax.scatter(xaxis, Y_test[:, ci], color=colors[ci], linewidth=0.1,
                       edgecolor='black', label=daytimes[ci].capitalize(), zorder=25)
            for y in range(Y_test.shape[0]):
                ax.fill_between([xaxis[y] - 0.25, xaxis[y] + 0.25],
                                Y_pred[y, ci] - rmse[daytimes[ci]],
                                Y_pred[y, ci] + rmse[daytimes[ci]],
                                color=colors[ci], alpha=0.33, edgecolor='black')
        ticks = np.arange(min(xaxis), max(xaxis) + 1, xres)
        tlabs = [datetime.strftime(ZERO_DATE + timedelta(days=int(t)),
                                   f"%d/%m\n%Y") for t in ticks]   
        ax.set_xticks(xaxis, minor=True)
        ax.set_xticks(ticks)
        ax.set_xticklabels(tlabs, fontsize=13, rotation=45)
        ax.set_ylabel('Number of incoming patients (NIP)', fontsize=13)
        ax.set_xlabel('Time (d/m/y)', fontsize=13)
        ax.set_xlim(xaxis[0] + (nx * (part - 1) / parts), xaxis[0] + (nx * part / parts))
        ax.set_title(f"Predicted vs Real NIP - {meth}")
        ax.legend(fontsize=14, loc='upper center', ncol=3, columnspacing=1)
        ax.set_yticks(np.arange(0, ylim + 1, 50))
        ax.set_yticks(np.arange(0, ylim + 1, 25), minor=True)
        ax.set_yticklabels(np.arange(0, ylim + 1, 50), fontsize=13)
        ax.set_ylim(0, ylim)
        plt.savefig(f"{tlpath}tl_{prefix}_ pt-{part}.pdf", bbox_inches='tight')
        plt.show()
        plt.close(figa)
    

# shift-based predictions
ycols = ['total_morning', 'total_afternoon', 'total_night']
timeLiner(X_TRAIN, X_TEST, Y_TRAIN[ycols], Y_TEST[ycols], aged=False, xres=7, parts=6)

# risk-group-based predictions
ycols = ['total_low', 'total_medium', 'total_high']
timeLiner(X_TRAIN, X_TEST, Y_TRAIN[ycols], Y_TEST[ycols], aged=True, xres=7, parts=6)

### Testing the models on post-COVID data
As explained in the paper, we only test SARIMA and RF

In [ ]:
# performance function
def perforMeter(pred_y, true_y, shifts=DAYTIMES):
    dicto = {'mape': {}, 'mape_std': {}, 'rmse': {},
             'rmse_std': {}, 'mae': {}, 'mae_std': {}}
    for t in shifts:
        diff = pred_y[t] - true_y[t]
        mape = 200 * np.mean(np.abs(diff) / (np.abs(pred_y[t]) + np.abs(true_y[t])), axis=1)
        rmse = np.sqrt(np.mean((diff) ** 2, axis=1))
        mae = np.mean(np.abs(diff), axis=1)
        dicto['mape'][t] = np.mean(mape)
        dicto['mape_std'][t] = np.std(mape)
        dicto['rmse'][t] = np.mean(rmse)
        dicto['rmse_std'][t] = np.std(rmse)
        dicto['mae'][t] = np.mean(mae)
        dicto['mae_std'][t] = np.std(mae)
    return dicto


# diebold fraction function
def dieboFunc(pred_a, pred_b, y_test, shifts=DAYTIMES, thresh=0.05, n_tests=1):
    diebolds = {}
    corr = thresh / n_tests
    for t in shifts:
        diebolds[t] = [dm_test(y_test[t][ni], pred_a[t][ni], pred_b[t][ni])[1]
                       for ni in range(y_test[t].shape[0])]
        diebolds[t] = [dm > corr for dm in diebolds[t]]
        diebolds[t] = np.sum(diebolds[t]) / len(diebolds[t])
    return diebolds


# Testing the models, generating the tables, and DM testing
def theCovidPredictor(X_train, X_test, Y_train, Y_test, ntrees,
                      niters=100, aged=False, arishift=28):
    T = {}
    wp_cols = ['timestep', 'weekday', 'yearday', 'holiday_-2',
               'holiday_-1', 'holiday_0', 'holiday_+1', 'holiday_+2',
               'temp_min', 'temp_max', 'prec_prob', 'wind_speed',
               'resident_pop', 'tourist_pop']
    wx_cols = ['timestep', 'weekday', 'yearday', 'holiday_-2',
               'holiday_-1', 'holiday_0', 'holiday_+1', 'holiday_+2',
               'temp_min', 'temp_max', 'prec_prob', 'wind_speed', 'resident_pop']
    xp_cols = ['timestep', 'weekday', 'yearday', 'holiday_-2',
               'holiday_-1', 'holiday_0', 'holiday_+1', 'holiday_+2',
               'resident_pop', 'tourist_pop']
    xx_cols = ['timestep', 'weekday', 'yearday', 'holiday_-2',
               'holiday_-1', 'holiday_0', 'holiday_+1', 'holiday_+2', 'resident_pop']
    if aged:
        daytimes = ['low', 'medium', 'high']
        prefix = 'risk'
    else:
        daytimes = DAYTIMES
        prefix = 'shift'
    # SARIMA
    Y_train, Y_test = Y_train.values, Y_test.values
    Y_a1 = np.zeros((len(Y_test), 3))
    Y_a7 = np.zeros((len(Y_test), 3))
    Y_a14 = np.zeros((len(Y_test), 3))
    for t in tqdm(np.arange(-14 + arishift, len(Y_test)-1), position=0, leave=True,
                  colour='red', ncols=111, desc='SARIMA'):
        Y = Y_test[:t+1]
        with warnings.catch_warnings(action="ignore"):
            arima = [ARIMA(Y[:, i], order=(1, 1, 1), seasonal_order=(1, 1, 1, 7),
                           ).fit() for i in range(3)]
        if t < len(Y_test) - 14:
            apred = np.array([arima[i].forecast(steps=14) for i in range(3)])
            if t > -2 + arishift:
                Y_a1[t+1] = apred[:, 0]
            if t > -8 + arishift:
                Y_a7[t+7] = apred[:, 6]
            Y_a14[t+14] = apred[:, 13]
        if t < len(Y_test) - 7:
            apred = np.array([arima[i].forecast(steps=7) for i in range(3)])
            Y_a1[t+1] = apred[:, 0]
            Y_a7[t+7] = apred[:, 6]
        else:
            apred = np.array([arima[i].forecast(steps=1)[0] for i in range(3)])
            Y_a1[t+1] = apred
    T['SARIMA-1'] = dict(zip(daytimes, np.array([Y_a1[:, t] for t in range(3)])))
    T['SARIMA-7'] = dict(zip(daytimes, np.array([Y_a7[:, t] for t in range(3)])))
    T['SARIMA-14'] = dict(zip(daytimes, np.array([Y_a14[:, t] for t in range(3)])))
    # RF
    X_tr = {'CWP': X_train[wp_cols].copy().values, 'CWX': X_train[wx_cols].copy().values,
            'CXP': X_train[xp_cols].copy().values, 'CXX': X_train[xx_cols].copy().values}
    T['RF'] = {}
    normies, woods = {}, {}
    for k, cols in zip(X_tr, [wp_cols, wx_cols, xp_cols, xx_cols]):
        normies[k] = [StandardScaler() for _ in range(X_tr[k].shape[1])]
        for i in range(len(normies[k])):
            normies[k][i].fit(X_tr[k][:, i].reshape(-1, 1))
            X_tr[k][:, i] = normies[k][i].transform(X_tr[k][:, i].reshape(-1, 1)).flatten()
        woods[k] = RandomForestRegressor(n_estimators=ntrees, n_jobs=-1) 
        woods[k].fit(X_tr[k], Y_train)
        T['RF'][k] = {}
        X_te = X_test[cols].copy().values
        for i in range(len(normies[k])):
            X_te[:, i] = normies[k][i].transform(X_te[:, i].reshape(-1, 1)).flatten()
        tree_preds = woods[k].predict(X_te)
        for t, dt in enumerate(daytimes):
            T['RF'][k][dt] = tree_preds[:, t]
    # Bootstrap procedure
    D = {'Y': dict(zip(daytimes, [[], [], []])),
         'SARIMA-1': dict(zip(daytimes, [[], [], []])),
         'SARIMA-7': dict(zip(daytimes, [[], [], []])),
         'SARIMA-14': dict(zip(daytimes, [[], [], []])),
         'RF': dict(zip(X_tr, [dict(zip(daytimes, [[], [], []])) for _ in range(4)])),}
    for ni in tqdm(range(niters), colour='red', ncols=111, desc='Bootstrapping'):
        if ni % 10 == 0:
            np.random.seed(SEED + ni // 10)
        vidx = np.random.randint(low=0, high=X_test.shape[0], size=X_test.shape[0])
        for t, dt in enumerate(daytimes):
            D['Y'][dt].append(Y_test[vidx, t])
            D['SARIMA-1'][dt].append(T['SARIMA-1'][dt][vidx])
            D['SARIMA-7'][dt].append(T['SARIMA-7'][dt][vidx])
            D['SARIMA-14'][dt].append(T['SARIMA-14'][dt][vidx])
            for k in X_tr:
                D['RF'][k][dt].append(T['RF'][k][dt][vidx])
    for t, dt in enumerate(daytimes):
        for meth in ['Y', 'SARIMA-1', 'SARIMA-7', 'SARIMA-14']:
            D[meth][dt] = np.array(D[meth][dt])
        for k in X_tr:
            D['RF'][k][dt] = np.array(D['RF'][k][dt])
    niters = len(D['SARIMA-1'])
    metrics = ['mape', 'rmse', 'mae']
    inpudict = {'CWP': 'All', 'CXP': 'No W', 'CWX': 'No T', 'CXX': 'No W T'}
    columns = ['method', 'input', 'shift', 'mape', 'mape_std',
               'rmse', 'rmse_std', 'mae', 'mae_std']
    # performance table
    df = []
    for meth in ['SARIMA-1', 'SARIMA-7', 'SARIMA-14']:
        dicto = perforMeter(D[meth], D['Y'], shifts=daytimes)
        for t in daytimes:
            df.append([meth, '', t.capitalize(), dicto['mape'][t], dicto['mape_std'][t],
                       dicto['rmse'][t], dicto['rmse_std'][t], dicto['mae'][t],
                       dicto['mae_std'][t]])
    for ind in inpudict:
        dicto = perforMeter(D['RF'][ind], D['Y'], shifts=daytimes)
        for t in daytimes:
            df.append(['RF', inpudict[ind], t.capitalize(), dicto['mape'][t],
                       dicto['mape_std'][t], dicto['rmse'][t], dicto['rmse_std'][t],
                       dicto['mae'][t], dicto['mae_std'][t]])
    df = pd.DataFrame(df, columns=columns)
    df.to_csv(f"covid-table-full_{prefix}.csv", index=None)
    display(df)
    # diebold Covid
    n_tests = 63
    dbar = tqdm(total=n_tests, colour='green', ncols=111,
                position=0, leave=True, desc=f'die-{prefix}')
    df = []
    columns = ['method_1', 'input_1', 'method_2', 'input_2', 'shift', 'equiv_frac']
    for m1, m2 in combinations(['SARIMA-1', 'SARIMA-7', 'SARIMA-14'], 2):
        diebold = dieboFunc(D[m1], D[m2], D['Y'],
                            shifts=daytimes, thresh=0.05, n_tests=n_tests)
        dbar.update(3)
        for t in daytimes:
            df.append([m1, '', m2, '', t.capitalize(), diebold[t]])
    for bmeth in ['SARIMA-1', 'SARIMA-7', 'SARIMA-14']:
        for ind in inpudict:
            diebold = dieboFunc(D[bmeth], D['RF'][ind], D['Y'],
                                shifts=daytimes, thresh=0.05, n_tests=n_tests)
            dbar.update(3)
            for t in daytimes:
                df.append([bmeth, '', 'RF', inpudict[ind], t.capitalize(), diebold[t]])
    for v1, v2 in combinations(['CWP', 'CWX', 'CXP', 'CXX'], 2):
        diebold = dieboFunc(D['RF'][v1], D['RF'][v2], D['Y'],
                            shifts=daytimes, thresh=0.05, n_tests=n_tests)
        dbar.update(3)
        for t in daytimes:
            df.append(['RF', inpudict[v1], 'RF', inpudict[v2], t.capitalize(), diebold[t]])
    dbar.close()
    df = pd.DataFrame(df, columns=columns)
    df.to_csv(f"covid-diebold_{prefix}.csv", index=None)
    display(df)
    
    
# shift-based predictions
ycols = ['total_morning', 'total_afternoon', 'total_night']
theCovidPredictor(X_TRAIN, X_COVID, Y_TRAIN[ycols], Y_COVID[ycols],
                  ntrees=100, niters=1000, aged=False)

# risk-group-based predictions
ycols = ['total_low', 'total_medium', 'total_high']
theCovidPredictor(X_TRAIN, X_COVID, Y_TRAIN[ycols], Y_COVID[ycols],
                  ntrees=100, niters=1000, aged=True)